<a href="https://colab.research.google.com/github/Emsidurito/M12AlbertG3DStudios/blob/main/Entrenador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cargar los csvs

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving Medicina_noticias.csv to Medicina_noticias.csv


Entrenador

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Obtener la lista de archivos CSV en la ubicación del código
csv_files = [file for file in os.listdir() if file.endswith('.csv')]

# DataFrame para almacenar todos los datos
all_data = pd.DataFrame(columns=['texto', 'categoria'])

# Leer y procesar cada archivo CSV
for csv_file in csv_files:
    # Obtener la categoría a partir del nombre del archivo CSV (antes del guion bajo)
    categoria = csv_file.split('_')[0]

    # Leer CSV y convertir a UTF-8
    df = pd.read_csv(csv_file, encoding='utf-8')

    # Asegurarse de que las columnas 'titulo' y 'noticia' existan en el DataFrame
    if 'Título' in df.columns and 'Noticia' in df.columns:
        # Combina 'titulo' y 'noticia' en una única columna 'texto'
        df['texto'] = df['Título'] + ' ' + df['Noticia']
        df['categoria'] = categoria
        # Concatenar al DataFrame general
        all_data = pd.concat([all_data, df[['texto', 'categoria']]], ignore_index=True)
    else:
        raise ValueError(f"Las columnas 'Titulo' y 'Noticia' no existen en el archivo {csv_file}.")

# Mezclar los datos
all_data = all_data.sample(frac=1).reset_index(drop=True)

# Verificar si existen categorías
categorias = set(all_data['categoria'])
if not categorias:
    raise ValueError("No se encontraron categorías en los datos.")

# Imprimir las categorías antes de continuar
print("Categorías:", categorias)

# Separar texto y etiquetas
X = all_data['texto']
y = all_data['categoria']

# Codificar las etiquetas y guardar el LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Tokenizar y hacer padding en el texto
tokenizer = Tokenizer()
X_train = X_train.dropna()
tokenizer.fit_on_texts(X_train)

# Guardar el Tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Aplicar tokenización y padding
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_seq)

# Crear el modelo de red neuronal con LSTM
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=X_train_padded.shape[1]))
model.add(LSTM(64, return_sequences=True))  # Capa LSTM con return_sequences=True
model.add(LSTM(64))  # Capa LSTM
model.add(Dense(len(set(y)), activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train_padded, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Guardar el modelo entrenado
model.save('modelo_entrenado.h5')


Categorías: {'Ocio', 'Informactica', 'Religion', 'Militar', 'Medicina', 'Motor', 'Moda', 'Economia', 'Astronomia', 'Alimentacion', 'Politica'}
Epoch 1/20
186/186 [==============================] - 108s 564ms/step - loss: 2.3258 - accuracy: 0.1511 - val_loss: 2.3199 - val_accuracy: 0.1526
Epoch 2/20
186/186 [==============================] - 104s 560ms/step - loss: 2.2431 - accuracy: 0.1894 - val_loss: 2.3816 - val_accuracy: 0.1465
Epoch 3/20
186/186 [==============================] - 104s 559ms/step - loss: 1.7420 - accuracy: 0.4181 - val_loss: 2.7592 - val_accuracy: 0.1196
Epoch 4/20
186/186 [==============================] - 100s 535ms/step - loss: 1.1058 - accuracy: 0.6652 - val_loss: 3.1718 - val_accuracy: 0.1169
Epoch 5/20
186/186 [==============================] - 96s 514ms/step - loss: 0.7952 - accuracy: 0.7637 - val_loss: 3.4481 - val_accuracy: 0.1069
Epoch 6/20
186/186 [==============================] - 97s 519ms/step - loss: 0.6366 - accuracy: 0.8025 - val_loss: 3.7765 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Descargar modelo entrenado

In [ ]:
from google.colab import files

# Reemplaza 'nombre_del_modelo.h5' con el nombre real de tu archivo h5
files.download('modelo_entrenado.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Programa de Prueba de clasificación

In [ ]:
import pickle
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tkinter import Tk, Label, Button, filedialog

# Cargar el modelo
modelo = load_model('modelo_entrenado.h5')

# Cargar el Tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Cargar el LabelEncoder
with open('label_encoder.pickle', 'rb') as handle:
    label_encoder = pickle.load(handle)

# Obtener la longitud máxima de la secuencia utilizada durante el entrenamiento
max_longitud = modelo.layers[0].input_shape[1]

# Función para preprocesar el texto
def preprocesar_texto(texto, tokenizer, max_longitud):
    secuencia = tokenizer.texts_to_sequences([texto])
    secuencia_pad = pad_sequences(secuencia, maxlen=max_longitud, padding='post')
    return secuencia_pad

# Función para realizar predicciones
def predecir_categoria(texto, modelo, tokenizer, max_longitud):
    secuencia_pad = preprocesar_texto(texto, tokenizer, max_longitud)
    predicciones = modelo.predict(secuencia_pad)
    categoria_predicha = label_encoder.inverse_transform([int(predicciones.argmax(axis=-1))])[0]
    return categoria_predicha

# Función para clasificar el contenido de un archivo
def clasificar_archivo():
    archivo_path = filedialog.askopenfilename(title="Selecciona un archivo de texto", filetypes=[("Text files", "*.txt")])

    if archivo_path:
        with open(archivo_path, 'r', encoding='utf-8') as file:
            contenido = file.read()
            categoria_predicha = predecir_categoria(contenido, modelo, tokenizer, max_longitud)
            etiqueta_resultado.config(text=f"Categoría predicha: {categoria_predicha}")

# Configuración de la interfaz gráfica
ventana = Tk()
ventana.title("Clasificador de Texto")

etiqueta_instrucciones = Label(ventana, text="Selecciona un archivo de texto para clasificar:")
etiqueta_instrucciones.pack()

boton_seleccionar = Button(ventana, text="Seleccionar Archivo", command=clasificar_archivo)
boton_seleccionar.pack()

etiqueta_resultado = Label(ventana, text="")
etiqueta_resultado.pack()

# Iniciar la interfaz gráfica
ventana.mainloop()

Programa de matriz de confusion

(La carpeta con el data set esta en el drive compartido)

In [ ]:
import os
import pickle
import pandas as pd
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import numpy as np

# Directorio que contiene los archivos CSV
directorio_data_test = 'data_test'

# Cargar el modelo
modelo = load_model('modelo_entrenado.h5')

# Cargar el Tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Cargar el LabelEncoder
with open('label_encoder.pickle', 'rb') as handle:
    label_encoder = pickle.load(handle)

# Definir max_longitud
max_longitud = modelo.layers[0].input_shape[1]

# Listas para almacenar las predicciones y las etiquetas reales
predicciones_totales = []
etiquetas_reales_totales = []

# Iterar sobre los archivos CSV en el directorio
for archivo_csv in os.listdir(directorio_data_test):
    if archivo_csv.endswith(".csv"):
        ruta_completa = os.path.join(directorio_data_test, archivo_csv)

        # Lee el CSV
        df = pd.read_csv(ruta_completa)

        # Asegurar que todos los valores en la columna 'Noticia' sean tratados como cadenas
        df['Noticia'] = df['Noticia'].astype(str)

        # Obtener el nombre de la categoría del nombre del archivo sin la extensión
        categoria = os.path.splitext(archivo_csv)[0]

        # Obtener las "Noticias" y realizar predicciones
        textos = df['Noticia'].tolist()
        textos_padded = pad_sequences(tokenizer.texts_to_sequences(textos), maxlen=max_longitud, padding='post')
        predicciones = label_encoder.inverse_transform(np.argmax(modelo.predict(textos_padded), axis=1))

        # Guardar las predicciones y las etiquetas reales
        predicciones_totales.extend(predicciones)
        etiquetas_reales_totales.extend([categoria] * len(predicciones))

# Calcular la matriz de confusión
matriz_confusion = confusion_matrix(etiquetas_reales_totales, predicciones_totales)
print("Matriz de Confusión:")
print(matriz_confusion)